In [ ]:
pasu = "crater_check.xlsx"
#データ読み込み
pd1 = pd.read_excel(pasu,sheet_name=0)

In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
CPPR    = model.inputDict['OBJRESH']
Rp      = Grid_size*CPPR
vimp    = -model.inputDict['OBJVEL']
vp      = vimp*1e-3
ts      = 2.*Rp/vimp *1e6
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

# Set up a pyplot figure with two axes. We can share the x (time) axes
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(234)
ax3 = fig.add_subplot(232)
ax4 = fig.add_subplot(235)
ax5 = fig.add_subplot(233)
ax6 = fig.add_subplot(236)

# Figure title
fig.suptitle('Al {} km/s, ts = {} us'.format(vp,ts),fontsize=15)

#ax1.set_title('Crater Growth',fontsize=20)
#ax3.set_title('Impact Point Thermokinetic \nParameters Growth',fontsize=20)
#ax5.set_title('Maximum Thermokinetic \nParameters(Projectile)',fontsize=20)
#ax7.set_title('Maximum Thermokinetic \nParameters (Destribution)',fontsize=20)

# Calculate crater dimensions through time
#last = 801
last = model.laststep
cg = model.craterGrowth(end=last)

# iSALE dimensions
radius = cg[:,2]
depth = -cg[:,3]
volume = cg[:,4]

# Convert experiment dimensions to mm
#rexpt *= 4.9
#dexpt *= 18.
rend = cg[last-1,2]
dend = -cg[last-1,3]
vend = cg[last-1,4]
# Convert time to us
time = cg[:,1]*1e6
#rtime *= 1e6
#dtime *= 1e6

# Plot iSALE crater dimensions
ax1.plot(time, radius, 'r--',label="iSALE")
ax2.plot(time, depth, 'b--',label="iSALE")
ax3.plot(time, volume, 'g--',label="iSALE")

# Plot experimental data
ax1.axhline(rend, color = "black", alpha = 0.2, lw=0.2, linestyle = "dashed",)
ax1.set_yticks([rend])
ax2.axhline(dend, color = "black", alpha = 0.2, lw=0.2, linestyle = "dashed")
ax2.set_yticks([dend])
ax3.axhline(vend, color = "black", alpha = 0.2, lw=0.2, linestyle = "dashed",)
ax3.set_yticks([vend])

ax1.set_yticklabels([' {: 5.1f} '.format(rend)])
ax2.set_yticklabels([' {: 5.1f} '.format(dend)])
ax3.set_yticklabels([' {: 5.1f} '.format(vend)])

def radius_f(x,r0,ra,rb):
        return r0 -  ra* np.exp(-rb*x)
r0_init = np.array([5,5,1])

def depth_f(x,d0,da,db):
        return d0 -  da* np.exp(-db*x)
d0_init = np.array([5,5,1])

def volume_f(x,v0,va,vb):
        return v0 -  va* np.exp(-vb*x)
v0_init = np.array([5,5,1])

time2 = time[0:last]
radius2 = radius[0:last]
depth2 = depth[0:last]
volume2 = volume[0:last]

time3 = np.arange(0,3000*ts,1*ts)

def fit(func, x, param_init):
        X = time2
        Y = x
        popt,pocv=curve_fit(func, X, Y, p0=param_init,maxfev=5000)
        perr = np.sqrt(np.diag(pocv))
        y=func(time3, *popt)
        return y, popt, perr

depth_est = fit(depth_f,depth2,d0_init)
radius_est = fit(radius_f,radius2,r0_init)
volume_est = fit(volume_f,volume2,v0_init)
dend2 = depth_est[1][0]
rend2 = radius_est[1][0]
vend2 = volume_est[1][0]

ax1.plot(time3, radius_est[0],'r--',linewidth=0.2, linestyle = "dashed",label="Regression")
ax2.plot(time3, depth_est[0], 'b--',linewidth=0.2, linestyle = "dashed",label="Regression")
ax3.plot(time3, volume_est[0], 'b--',linewidth=0.2, linestyle = "dashed",label="Regression")

ax1.axhline(rend2, color = "black", alpha = 0.2, linewidth=0.2, linestyle = "dashed")
ax2.axhline(dend2, color = "black", alpha = 0.2, linewidth=0.2, linestyle = "dashed")
ax3.axhline(vend2, color = "black", alpha = 0.2, linewidth=0.2, linestyle = "dashed")


rlabelscale = len(str(rend2).split('.')[0])
rlabel1 = 10**(rlabelscale)
rlabel2 = rlabel1/2
dlabelscale = len(str(dend2).split('.')[0])
dlabel1 = 10**(dlabelscale)
dlabel2 = dlabel1/2
vlabelscale = len(str(vend2).split('.')[0])
vlabel1 = 10**(vlabelscale)
vlabel2 = vlabel1/2

if rend2 <= rlabel2:
        rlabel1 = rlabel2
        rlabel2 = float(rlabel2)/2
if dend2 <= dlabel2:
        dlabel1 = dlabel2
        dlabel2 = float(dlabel2)/2
if vend2 <= vlabel2:
        vlabel1 = vlabel2
        vlabel2 = float(vlabel2)/2


ax1.set_yticklabels([rlabel1,rlabel2,' {: 5.1f} '.format(rend2)])
ax2.set_yticklabels([dlabel1,dlabel2,' {: 5.1f} '.format(dend2)])
ax3.set_yticklabels([vlabel1,vlabel2,' {: 5.1f} '.format(vend2)])

ax1.set_yticks([rlabel1,rlabel2,rend2])
ax2.set_yticks([dlabel1,dlabel2,dend2])
ax3.set_yticks([vlabel1,vlabel2,vend2])
# Set axes limits
ax1.set_xlim(0,50*ts)
ax2.set_xlim(0,50*ts)
ax3.set_xlim(0,50*ts)
#ax1.set_ylim(0,10)
#ax2.set_ylim(0,10)



ax1.set_yticks([radius[last-1]])
ax1.set_yticklabels([' {: 5.1f} '.format(radius[last-1])])
ax2.set_yticks([depth[last-1]])
ax2.set_yticklabels([' {: 5.1f} '.format(depth[last-1])])
ax3.set_yticks([volume[last-1]])
ax3.set_yticklabels([' {: 5.1f} '.format(volume[last-1])])

# Set axes labels
ax1.set_xlabel('Time [$\mu$s]')
ax2.set_xlabel('Time [$\mu$s]')
ax3.set_xlabel('Time [$\mu$s]')
ax1.set_ylabel('Crater Radius [mm]')
ax2.set_ylabel('Crater Depth [mm]')
ax3.set_ylabel('Crater volume [mm3]')


# Axes legends
ax1.legend(loc="upper right", bbox_to_anchor=(0.99,0.99), borderaxespad=0, fontsize=10)
ax2.legend(loc="upper right", bbox_to_anchor=(0.99, 0.99), borderaxespad=0, fontsize=10)
ax3.legend(loc="upper right", bbox_to_anchor=(0.99, 0.99), borderaxespad=0, fontsize=10)
ax1.text(0.98,0.02,"R = r0 - ra* exp(-rb*t)\n r0= {} $\pm$ {}\nra= {} $\pm$ {}\nrb= {} $\pm$ {}".format(round(radius_est[1][0],2),round(radius_est[2][0],2),round(radius_est[1][1],2),round(radius_est[2][1],2),round(radius_est[1][2],2),round(radius_est[2][2],2)),color="g", fontsize=10, va='bottom', ha='right',transform=ax1.transAxes)
ax2.text(0.98,0.02,"D = d0 - da* exp(-db*t)\n d0= {} $\pm$ {}\nda= {} $\pm$ {}\ndb= {} $\pm$ {}".format(round(depth_est[1][0],2),round(depth_est[2][0],2),round(depth_est[1][1],2),round(depth_est[2][1],2),round(depth_est[1][2],2),round(depth_est[2][2],2)),color="g", fontsize=10, va='bottom', ha='right',transform=ax2.transAxes)
ax3.text(0.98,0.02,"V = v0 - va* exp(-vb*t)\n v0= {} $\pm$ {}\nva= {} $\pm$ {}\nvb= {} $\pm$ {}".format(round(volume_est[1][0],2),round(volume_est[2][0],2),round(volume_est[1][1],2),round(volume_est[2][1],2),round(volume_est[1][2],2),round(volume_est[2][2],2)),color="g", fontsize=10, va='bottom', ha='right',transform=ax3.transAxes)


step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT'],model.laststep)

time2 = []
under_maxp = []
all_maxp = []
under_maxt = []
all_maxt = []
under_maxd = []
all_maxd = []

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x3 = x2[x2  < num_impactor ]
x3x = step0.xmark[x3]
x3y = step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
x3T = step1.TrT[x3]
maxp1 = max(x3P[0:20])
maxt1 = max(x3T[0:20])
maxP1 =round(maxp1,1)
maxT1 =round(maxt1,0)
maxp2 = max(x3P)
maxt2 = max(x3T)
maxP2 =round(maxp2,1)
maxT2 =round(maxt2,0)


hugo = np.genfromtxt('./Ice/SETUP/hugoniot-H2O_ice-h2o_ice-aneos.txt',usecols=(0,3,4))
tr1 = np.where((step0.xmark <= Grid_size*1e3/2)&(step0.ymark <= 0)&(step0.ymark > -Grid_size*1e3))
tracer_id = tr1[0][0]
#if(len(tracer_id)>1):
#       tracer_id = tracer_id[0]
Tracer_R = []
Tracer_Z = []
Tracer_Den = []
Tracer_Pre = []
Tracer_Tmp = []

#last= 5
for i in np.arange(0,last,1):
        time2.append(dt*i)

        step   = model.readStep(['Trp','Trt','Trd'],i)
        all_p = step.Trp*1e-9
        under_p = step.Trp[x2]*1e-9
        all_maxp.append(max(all_p))
        under_maxp.append(max(under_p))

        all_t = step.Trt
        under_t = step.Trt[x2]
        all_maxt.append(max(all_t))
        under_maxt.append(max(under_t))

        all_d = step.Trd
        under_d = step.Trd[x2]
        all_maxd.append(max(all_d))
        under_maxd.append(max(under_d))

        Tracer_R.append(step.xmark[tracer_id])
        Tracer_Z.append(step.ymark[tracer_id])
        Tracer_Den.append(step.Trd[tracer_id])
        Tracer_Pre.append(step.Trp[tracer_id]*1.0e-9)
        Tracer_Tmp.append(step.Trt[tracer_id])

max_all_maxp = max(Tracer_Pre)
hugo_st = np.argmax(Tracer_Pre)
max_all_maxd = Tracer_Den[hugo_st]
max_all_maxt = Tracer_Tmp[hugo_st]

prelabelscale = len(str(max_all_maxp).split('.')[0])
prelabel1 = 10**(prelabelscale)
prelabel2 = prelabel1/2
tmplabelscale = len(str(max_all_maxt).split('.')[0])
tmplabel1 = 10**(tmplabelscale)
tmplabel2 = tmplabel1/2
denlabelscale = len(str(max_all_maxd).split('.')[0])
denlabel1 = 10**(denlabelscale)
denlabel2 = denlabel1/2

if max_all_maxp <= prelabel2:
        prelabel1 = prelabel2
        prelabel2 = float(prelabel2)/2
if max_all_maxt <= tmplabel2:
        tmplabel1 = tmplabel2
        tmplabel2 = tmplabel2/2
if max_all_maxd <= denlabel2:
        denlabel1 = denlabel2
        denlabel2 = denlabel2/2

hugo_d = hugo[:,0].tolist()
hugo_p0 = hugo[:,1]*1e-9
hugo_p = hugo_p0.tolist()

ax4.plot(Tracer_Den,Tracer_Pre, Color='black', linestyle='solid', linewidth = 3.0,zorder=1)
ax4.plot(hugo_d,hugo_p,color='red', linestyle='solid', linewidth = 3.0, zorder =1)
# set axis labels
ax4.set_xlabel('Density [kg/m3]')
ax4.set_ylabel('Pressure [GPa]')


# set axis limits
ax4.axvline(max_all_maxd, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax3.legend(fontsize=8)
ax4.set_xticks([max_all_maxd,denlabel1])
ax4.set_xticklabels([' {: 5.1f} '.format(max_all_maxd),denlabel1])
ax4.set_xlim([0,denlabel1])

# set axis limits
ax4.axhline(max_all_maxp, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax3.legend(fontsize=8)
ax4.set_yticks([prelabel2,max_all_maxp,prelabel1])
ax4.set_yticklabels([prelabel2,' {: 5.1f} '.format(max_all_maxp),prelabel1])
ax4.set_ylim([0,prelabel1])

ax5.plot(x3y, x3P, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure")
# set axis labels
ax5.set_xlabel('z [mm]')
ax5.set_ylabel('Pressure [GPa]')
ax5.axhline(maxp1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax5.axhline(maxp2, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax5.set_yticks([prelabel2,prelabel1,maxp1,maxp2])
ax5.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1),' {: 5.1f} '.format(maxP2)])
# set axis limits
ax5.set_xlim([0,1])
#ax5.set_aspect ('equal')
#ax5.tick_params(axis='y', colors=color_1)

ax6.plot(x3y, x3T,  linewidth = 0.2, c= "red",zorder=1,label="Peak Temperature")
# set axis labels
ax6.set_ylabel('Temperature [K]')
ax6.set_xlabel('z  [mm]')
ax6.axhline(maxt1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax6.axhline(maxt2, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax6.tick_params(axis='y', colors=color_2)
ax6.set_yticks([tmplabel2,tmplabel1,maxt1,maxt2])
ax6.set_yticklabels([tmplabel2,tmplabel1,' {: 5.1f} '.format(maxT1),' {: 5.1f} '.format(maxT2)])
# set axis limits
ax6.set_xlim([0,1])
#ax5.grid(True)
#ax6.grid(True)
#ax6.tick_params(axis='y', colors=color_2)



fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)
plt.savefig('../property_png/{}property.png'.format(dirname))
plt.savefig('./{}property.png'.format(dirname))

dataset =[['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]'],
          [dirname,vp,ts, 2*radius_est[1][0],depth_est[1][0],volume_est[1][0],max_all_maxp,max_all_maxt,max_all_maxd]]
with open('../property_csv/{}property.csv'.format(dirname), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(dataset)
    f.close()
with open('./{}property.csv'.format(dirname), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(dataset)
    f.close()

In [1]:
import csv
import pandas as pd

df_all_ANEOS = [['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]']]
df_all_ANEHY = [['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]']]
df_all_TILLO = [['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]']]
df_all_TILHY = [['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]']]
df_all_Kato = [['File name','velocity [km/s]','ts [us]','Crater Dia. [mm]','Crater Dep.[mm]','Crater Vol. [mm3]','Peak pre. [GPa]','Peak tmp. [K]','Peak den. [kg/m3]']]

#csvファイルの読み込み
for i in np.arange(1,17,1):
  folder_name = 'aliceANEOS'+str(i)
  df_ANEOS = pd.read_csv('./{}property.csv'.format(folder_name))[1]
  df_all_ANEOS.append(df_ANEOS)

for i in np.arange(1,17,1):
  folder_name = 'aliceANEHY'+str(i)
  df_ANEHY = pd.read_csv('./{}property.csv'.format(folder_name))[1]
  df_all_ANEHY.append(df_ANEHY)

for i in np.arange(1,17,1):
  folder_name = 'aliceTILLO'+str(i)
  df_TILLO = pd.read_csv('./{}property.csv'.format(folder_name))[1]
  df_all_TILLO.append(df_TILLO)

for i in np.arange(1,17,1):
  folder_name = 'aliceTILHY'+str(i)
  df_TILHY = pd.read_csv('./{}property.csv'.format(folder_name))[1]
  df_all_TILHY.append(df_TILHY)

for i in np.arange(1,7,1):
  folder_name = 'Kato'+str(i)
  df_Kato = pd.read_csv('./{}property.csv'.format(folder_name))[1]
  df_all_Kato.append(df_Kato)

#データの整理
df_dia = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,3],df_all_ANEHY[1:,3],df_all_TILLO[1:,3],df_all_TILHY[1:,3]]
df_dia.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

df_rad = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,4],df_all_ANEHY[1:,4],df_all_TILLO[1:,4],df_all_TILHY[1:,4]]
df_rad.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

df_vol = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,5],df_all_ANEHY[1:,5],df_all_TILLO[1:,5],df_all_TILHY[1:,5]]
df_vol.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

df_pre = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,6],df_all_ANEHY[1:,6],df_all_TILLO[1:,6],df_all_TILHY[1:,6]]
df_pre.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

df_tmp = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,7],df_all_ANEHY[1:,7],df_all_TILLO[1:,7],df_all_TILHY[1:,7]]
df_tmp.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

df_den = [df_all_ANEOS[1:,1],df_all_ANEOS[1:,8],df_all_ANEHY[1:,8],df_all_TILLO[1:,8],df_all_TILHY[1:,8]]
df_den.columns = ['velocity [km/s]','ANEOS','ANEHY','TILLO','TILHY']

#データの出力
df_all_ANEOS.to_csv('./df_all_ANEOS.csv')
df_all_ANEHY.to_csv('./df_all_ANEHY.csv')
df_all_TILLO.to_csv('./df_all_TILLO.csv')
df_all_TILHY.to_csv('./df_all_TILHY.csv')
df_all_Kato.to_csv('./df_all_Kato.csv')
df_dia.to_csv('./df_dia.csv')
df_rad.to_csv('./df_dia.csv')
df_vol.to_csv('./df_dia.csv')
df_pre.to_csv('./df_pre.csv')
df_tmp.to_csv('./df_tmp.csv')
df_den.to_csv('./df_den.csv')




NameError: name 'np' is not defined

In [22]:
#解析準備
#module load anaconda/2 intel
#ipython起動
#ipython --matplotlib
# prompt: フォルダを削除し、シンボリックリンクを作成
import numpy as np
import os
import shutil

for i in np.arange(1,17,1):
  folder_name_ANEOS = 'aliceANEOS'+str(i)
  folder_name_ANEHY = 'aliceANEHY'+str(i)
  folder_name_TILLO = 'aliceTILLO'+str(i)
  folder_name_TILHY = 'aliceTILHY'+str(i)

  shutil.rmtree('/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_ANEOS))
  shutil.rmtree('/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_ANEHY))
  shutil.rmtree('/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_TILLO))
  shutil.rmtree('/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_TILHY))


  os.symlink('/home/nakazawajn/isale-work/share/examples/Kato1/Plotting', '/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_ANEOS))
  os.symlink('/home/nakazawajn/isale-work/share/examples/Kato1/Plotting', '/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_ANEHY))
  os.symlink('/home/nakazawajn/isale-work/share/examples/Kato1/Plotting', '/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_TILLO))
  os.symlink('/home/nakazawajn/isale-work/share/examples/Kato1/Plotting', '/home/nakazawajn/isale-work/share/examples/{}/Plotting'.format(folder_name_TILHY))



aliceANEOS1
aliceANEOS2
aliceANEOS3
aliceANEOS4
aliceANEOS5
aliceANEOS6
aliceANEOS7
aliceANEOS8
aliceANEOS9
aliceANEOS10
aliceANEOS11
aliceANEOS12
aliceANEOS13
aliceANEOS14
aliceANEOS15
aliceANEOS16


In [1]:
import numpy as np
import subprocess
import os

for i in np.arange(1,17,1):
  folder_name = 'aliceANEOS'+str(i)
  qsub_name   = 'ANEOS'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceANEHY'+str(i)
  qsub_name   = 'ANEHY'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceTILLO'+str(i)
  qsub_name   = 'TILLO'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceTILHY'+str(i)
  qsub_name   = 'TILHY'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])


FileNotFoundError: [Errno 2] No such file or directory: '/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh' -> '/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh'

In [1]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('m')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
CPPR    = model.inputDict['OBJRESH']
Rp      = Grid_size*CPPR
vimp    = -model.inputDict['OBJVEL']
vp      = vimp
ts      = 2.*Rp/vimp
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

step0   = model.readStep(['Trp','Trd'],0)
#Tr1 -> Target、Tr2 -> projectile
tr1 = np.where((step0.xmark <= Grid_size*1/2)&(step0.ymark <= 0)&(step0.ymark > -Grid_size))
tr1 = tr1[0][0]
tr2 = 0
ut=[]
up=[]
pret=[]
prep=[]

for i in np.arange(0,model.laststep):
  step1   = model.readStep(['Trp','Trd'],i+1)

  vx1 = (step1.xmark[tr1]-step0.xmark[tr1])/dt
  vy1 = (step1.ymark[tr1]-step0.ymark[tr1])/dt
  vx2 = (step1.xmark[tr2]-step0.xmark[tr2])/dt
  vy2 = (step1.ymark[tr2]-step0.ymark[tr2])/dt

  v1 = ((vx1)**2 + (vy1)**2)**0.5
  v2 = ((vx2)**2 + (vy2)**2)**0.5


  ut.append(vy1)
  up.append(vy2)
  pret.append(step1.Trp[tr1]*1e-6)
  prep.append(step1.Trp[tr2]*1e-6)

  step0 = step1

ModuleNotFoundError: No module named 'pySALEPlot'

In [ ]:

# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('m')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
CPPR    = model.inputDict['OBJRESH']
Rp      = Grid_size*CPPR
vimp    = -model.inputDict['OBJVEL']
vp      = vimp
ts      = 2.*Rp/vimp
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)
ax6 = fig.add_subplot(236)

# Figure title
fig.suptitle('Al {} km/s, ts = {} us'.format(vp,ts),fontsize=15)


step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)



x3x = step0.xmark[x3]*1e3
x3y = -step0.ymark[x3]*1e3
x3P = step1.TrP[x3]*1e-9
#x3D = step1.TrD[x3]
x3T = step1.TrT[x3]
maxp1 = max(x3P)
maxt1 = max(x3T)
#maxd1 = max(x3D)
maxP1 =round(maxp1,1)
#maxD1 =round(maxd1,0)
maxT1 =round(maxt1,0)
d_r=x3y/(Rp*1e3)

prelabelscale = len(str(maxp1).split('.')[0])
prelabel1 = 10**(prelabelscale)
prelabel2 = prelabel1/2
tmplabelscale = len(str(maxt1).split('.')[0])
tmplabel1 = 10**(tmplabelscale)
tmplabel2 = tmplabel1/2
#denlabelscale = len(str(maxd1).split('.')[0])
#denlabel1 = 10**(denlabelscale)
#denlabel2 = denlabel1/2

if maxp1 <= prelabel2:
        prelabel1 = prelabel2
        prelabel2 = float(prelabel2)/2
if maxt1 <= tmplabel2:
        tmplabel1 = tmplabel2
        tmplabel2 = tmplabel2/2
#if maxd1 <= denlabel2:
#        denlabel1 = denlabel2
#        denlabel2 = denlabel2/2

maxy1 = max(x3y)
ax1.clear()
ax1.set_yscale('log')  # y軸をlogスケールで描く
ax1.set_xscale('log')  # x軸をlogスケールで描く
ax1.plot(x3y,x3P, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure")
# set axis labels
ax1.set_xlabel('d/R ')
ax1.set_ylabel('Pressure [GPa]')
ax1.axhline(maxp1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax1.axvline(maxy1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax1.set_xlim([0.01,10])
ax1.set_ylim([1,100])

#ax1.set_yticks([prelabel2,prelabel1,maxp1])
#ax1.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1)])
# set axis limits
#ax1.set_aspect ('equal')
#ax1.tick_params(axis='y', colors=color_1)

#ax2.plot(x3y, x3D,  linewidth = 0.2, c= "red",zorder=1,label="Peak Temperature")
# set axis labels
ax2.set_ylabel('Density [kg/m3]')
ax2.set_xlabel('d  [mm]')
#ax2.axhline(maxd1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax2.axhline(maxd2, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax6.tick_params(axis='y', colors=color_2)
ax2.set_yticks([tmplabel2,tmplabel1,maxt1])
ax2.set_yticklabels([tmplabel2,tmplabel1,' {: 5.1f} '.format(maxT1)])
# set axis limits
ax2.set_xlim([0,1])
#ax5.grid(True)
#ax6.grid(True)
#ax6.tick_params(axis='y', colors=color_2)

ax3.plot(x3y, x3T,  linewidth = 0.2, c= "red",zorder=1,label="Peak Temperature")
# set axis labels
ax3.set_ylabel('Temperature [K]')
ax3.set_xlabel('d  [mm]')
ax3.axhline(maxt1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax3.axhline(maxt2, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax6.tick_params(axis='y', colors=color_2)
ax3.set_yticks([tmplabel2,tmplabel1,maxt1])
ax3.set_yticklabels([tmplabel2,tmplabel1,' {: 5.1f} '.format(maxT1)])
# set axis limits
ax3.set_xlim([0,1])


#cb1.remove()
#ax4.remove()
#ax4 = fig.add_subplot(234)
# Plot the pressure field
for u in range(model.tracer_numu): #loop over tracer clouds
        tstart = model.tru[u].start
        tend = model.tru[u].end
        scat1 = ax4.scatter(step0.xmark[tstart:tend]*1e3,step0.ymark[tstart:tend]*1e3,
                c=step1.TrP[tstart:tend]*1e-9,vmin=maxp1/8,vmax=maxp1/4.5,
                cmap='viridis',s=4,linewidths=0)
cb1=fig.colorbar(scat1,ax=ax4)
cb1.set_label('Tracer Prak Pressure [GPa]')
ax4.set_xlabel('x  [mm]')
ax4.set_ylabel('z  [mm]')
ax4.set_xlim([0,1.5])
ax4.set_ylim([-1.5,1.1])



# Plot the density field
# for u in range(model.tracer_numu): #loop over tracer clouds
#         tstart = model.tru[u].start
#         tend = model.tru[u].end
#         scat2 = ax5.scatter(step0.xmark[tstart:tend]*1e3,step0.ymark[tstart:tend]*1e3,
#                 c=step1.TrD[tstart:tend],vmin=0.9,vmax=1.5,
#                 cmap='viridis',s=4,linewidths=0)
# cb2=fig.colorbar(scat2,ax=ax5)
# cb2.set_label('Tracer Prak Density [K]')
# ax5.set_xlabel('x  [mm]')
# ax5.set_ylabel('z  [mm]')
# ax5.set_xlim([0,1.5])
# ax5.set_ylim([-2,1.1])

#cb3.remove()
#ax6.remove()
#ax6 = fig.add_subplot(236)
# Plot the tempareture field
for u in range(model.tracer_numu): #loop over tracer clouds
        tstart = model.tru[u].start
        tend = model.tru[u].end
        scat3 = ax6.scatter(step0.xmark[tstart:tend]*1e3,step0.ymark[tstart:tend]*1e3,
                c=step1.TrT[tstart:tend],vmin=(maxt1-263)/8+263,vmax=(maxt1-263)/4.5+263,
                cmap='viridis',s=4,linewidths=0)
cb3=fig.colorbar(scat3,ax=ax6)
cb3.set_label('Tracer Prak Tempareture [K]')
ax6.set_xlabel('x  [mm]')
ax6.set_ylabel('z  [mm]')
ax6.set_xlim([0,1.5])
ax6.set_ylim([-1.5,1.1])


fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}property.png'.format(dirname))



In [ ]:
import numpy as np
import subprocess
import os

for i in np.arange(1,17,1):
  folder_name = 'aliceANEOS'+str(i)
  qsub_name   = 'ANEOS'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceANEHY'+str(i)
  qsub_name   = 'ANEHY'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceTILLO'+str(i)
  qsub_name   = 'TILLO'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])

  folder_name = 'aliceTILHY'+str(i)
  qsub_name   = 'TILHY'+str(i)
  os.symlink('/home/nakazawajn/isale-work/share/examples/aliceANEOS1/analysis.sh', '/home/nakazawajn/isale-work/share/examples/{}/analysis.sh'.format(folder_name))
  subprocess.run(['cd','./{}'.format(folder_name)])
  subprocess.run(['qsub','-N','{}'.format(qsub_name),'analysis.sh'])
  subprocess.run(['cd','../'])